In [103]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.error import HTTPError 
from urllib.error import URLError
import time
import pandas as pd
import numpy as np

try:
    html = urlopen('http://www.pythonscraping.com/pages/page1.html') 
except HTTPError as e: 
    print(e)
except URLError as e:
    print('The server could not be found!')
else:
    print('It Worked!')
    bs = BeautifulSoup(html.read(), 'html.parser')
    print(bs.h1)

It Worked!
<h1>An Interesting Title</h1>


In [104]:
from selenium import webdriver
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome("/usr/local/bin/chromedriver", chrome_options=options)

<ipython-input-104-2c6674e248ce>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/local/bin/chromedriver", chrome_options=options)
<ipython-input-104-2c6674e248ce>:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome("/usr/local/bin/chromedriver", chrome_options=options)


In [127]:
food_and_drink_links = dict()
print(food_and_drink_links)

{}


In [106]:
# Robot clicking load more in food and drinks

import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

wait = WebDriverWait(driver, 5)
driver.get("https://curiocity.com/category/food-drink/?city=toronto")
for x in range(60):
        button = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@class='curiocity-load-more link link--underlined']")))
        if button.is_displayed():
            driver.execute_script("arguments[0].click();", button)
            time.sleep(3)
page_source = driver.page_source

In [107]:
from bs4 import BeautifulSoup

# This method is getting the html of the website and gathering all the links
# of the posts in a list (food_and_drink_links)
soup = BeautifulSoup(page_source, 'lxml')
cards = []
cards_selector = soup.find_all('article', class_='card')
for card_selector in cards_selector:
    header = card_selector.h3
    a = header.find('a', class_ ='card__link')
    link = a['href']
    title = a.text
    title = title.lstrip()
    food_and_drink_links[title] = link 

In [108]:
# Check the html architecture
driver.get(cards_selector[5].h3.a["href"])
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [109]:

card_1item = soup.find('main', class_="grid__item").findAll('h3')
TITLE = soup.find('div', class_="card__header") # This is the title of the card (is not especial to the ONE_CARD)
TITLE = TITLE.h1.text.strip('\n')
CATEGORY = soup.find('dl', class_="post__category").dd.a.text
tags = soup.find('div', class_="post__tags")
if tags == None:
    print("1")
#     TAGS_NAMES = ""
LINK = soup.find('figure', class_="wide").img["src"]
#     for tag in tags:
#         TAGS_NAMES += (tag.text + ", ")
#     TAGS_NAMES
DESCRIPTION = soup.find('div', class_="post-content grid__item")
DESCRIPTION = DESCRIPTION.findAll("p")
DESCRIPTION_FULL = ""
WHERE = ""
WHEN = ""
for p in DESCRIPTION:
    if 'Where:' in p.text :
        atts = p.text
        try :
            atts = atts.split("\n")
        except :
            print('Where and when not breakable')
        else :
            WHERE = atts[0][6:].lstrip(" ")
            WHEN = atts[1][5:].lstrip(" ")
    else :
        DESCRIPTION_FULL += (p.text + " ")
TITLE

"Toronto's Moscow Tea Room is getting revamped with a new name & theme"

In [110]:
len(food_and_drink_links)

488

# This is the first web scrapper for food and drinks

## This is going to be an approach of decision tree
1. first we are going to check the posts with one card and store the other links in a "to scrap" list

In [112]:
scrapped = []
to_scrap = []

In [114]:
# Scrapper of cards with just one item
naive_card_db = {'title':[], 'description':[], 'when':[], 'where':[], 'link':[], 'category':[], 'tags':[]}
naive_card_db = pd.DataFrame(naive_card_db)
for link in food_and_drink_links:
    driver.get(food_and_drink_links[link])
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    card_1item = soup.find('main', class_="grid__item").findAll('h3')
    if (len(card_1item) == 1) | (link in scrapped):
        TITLE = soup.find('div', class_="card__header") # This is the title of the card (is not especial to the ONE_CARD)
        TITLE = TITLE.h1.text.strip('\n')
        LINK = soup.find('figure', class_="wide").img["src"]
        CATEGORY = soup.find('dl', class_="post__category").dd.a.text
        tags = soup.find('div', class_="post__tags")
        TAGS_NAMES = ""
        if (tags != None):
            tags = tags.findAll('a')
            for tag in tags:
                TAGS_NAMES += (tag.text + ", ")
        DESCRIPTION = soup.find('div', class_="post-content grid__item")
        DESCRIPTION = DESCRIPTION.findAll("p")
        DESCRIPTION_FULL = ""
        WHERE = ""
        WHEN = ""
        for p in DESCRIPTION:
            if 'Where:' in p.text :
                atts = p.text
                try :
                    atts = atts.split("\n")
                except :
                    print('Where and when not breakable')
                else :
                    if (len(atts) >= 2):
                        WHERE = atts[0][6:].lstrip(" ")
                        WHEN = atts[1][5:].lstrip(" ")
                        row = {'title':TITLE, 'description':DESCRIPTION_FULL, 'when':WHEN, 'where':WHERE, 'link':LINK, 'category':CATEGORY}
                        naive_card_db = naive_card_db.append(row, ignore_index=True)
                    else :
                        row = {'title':TITLE, 'description':DESCRIPTION_FULL, 'when':"n/a", 'where':atts[0][6:], 'link':LINK, 'category':CATEGORY}
                        naive_card_db = naive_card_db.append(row, ignore_index=True)
                        print(atts)
            else :
                DESCRIPTION_FULL += (p.text + " ")
        print(TITLE)
        scrapped.append(link)
    else :
        to_scrap.append(link)
    time.sleep(2)
    
    

Burger Drops is giving out FREE burgers this week & here's where you can get one
Toronto's new enchanted tea garden pop-up is the place to be this spring
Toronto's Moscow Tea Room is getting revamped with a new name & theme
Toronto's massive Rosé Picnic is back this summer & here's what to expect
A Pokémon Maid Cafe pop-up is coming to Toronto this weekend
You can get all-you-can-eat cheeseburgers in Toronto with this new 'Gold Card'
Toronto's getting an all-inclusive Italian fest with unlimited food, drink & entertainment
Toronto has a new late-night spot inspired by Barcelona's laneway tapas bars
World's largest BBQ concept Dickey's is opening in Toronto this summer
Toronto's getting a raccoon-themed bar & it's as amazing as it sounds
Canada's first Casa Madera is opening in Toronto & you can get your drinks lit on fire
Toronto's getting a two-hour 'unlimited' wine & cheese fest this April
Toronto bakery hiding treasure chests with prizes & an egg stuffed with $1,000
Toronto's gettin

In [115]:
naive_card_db

,title,description,when,where,link,category,tags
0,Burger Drops is giving out FREE burgers this w...,Toronto’s fave smash burger spot is giving out...,"Friday April 29, 2022 from 11:30am-6pm",Liberty House Apartments – 15 Solidarity Way,https://curiocity.com/wp-content/uploads/2022/...,Food & Drink,NaN
1,Toronto's new enchanted tea garden pop-up is t...,Celebrate springtime in the speakeasy era at t...,"Until July 31, 2022",325 Bay St,https://curiocity.com/wp-content/uploads/2022/...,Food & Drink,NaN
2,Toronto's Moscow Tea Room is getting revamped ...,It looks like it’s the start of a new era for ...,"Set to open May 12, 2022",161 Yorkville Ave,https://curiocity.com/wp-content/uploads/2022/...,Food & Drink,NaN
3,Toronto's massive Rosé Picnic is back this sum...,The biggest picnic of the summer is returning ...,"August 6, 2022","Bandshell Park, 100 Prince Edward Island Cres",https://curiocity.com/wp-content/uploads/2022/...,Food & Drink,NaN
4,A Pokémon Maid Cafe pop-up is coming to Toront...,"Gotta catch ’em all! Pokémon super fans, you w...","April 23 & 24, 2022","Oishiii Sweets, 3376 Kennedy Rd, Unit 2",https://curiocity.com/wp-content/uploads/2022/...,Food & Drink,NaN
...,...,...,...,...,...,...,...
176,A highly-anticipated Thai spot just opened in ...,The wait is over on a brand new second locatio...,n/a,,https://curiocity.com/wp-content/uploads/2021/...,Food & Drink,NaN
177,Embrace the cold with vegan ice cream from thi...,There are two types of people in the world. Pe...,n/a,,https://curiocity.com/wp-content/uploads/2021/...,Food & Drink,NaN
178,"The new ""EatingYYZ"" connects Toronto with all ...","Eating local has never been more important, an...",n/a,,https://curiocity.com/wp-content/uploads/2021/...,Food & Drink,NaN
179,One of Toronto's fave places for sky-high burg...,"Listen up foodies, we’ve got great news. A bra...",n/a,,https://curiocity.com/wp-content/uploads/2021/...,Food & Drink,NaN


In [118]:
result = naive_card_db
result.to_csv('~/Documents/INFO/food_drinks_toronto_card_1.csv', index=True)

In [120]:
result.to_json('~/Documents/INFO/food_drinks_toronto_card_1.json', orient='table')

# Getting card 1 data for vancouver

In [138]:
from selenium import webdriver
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome("/usr/local/bin/chromedriver", chrome_options=options)

<ipython-input-138-2c6674e248ce>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/local/bin/chromedriver", chrome_options=options)
<ipython-input-138-2c6674e248ce>:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome("/usr/local/bin/chromedriver", chrome_options=options)


In [139]:
food_and_drink_links = dict()
print(food_and_drink_links)

{}


In [140]:
wait = WebDriverWait(driver, 5)
driver.get("https://curiocity.com/category/food-drink/?city=vancouver")
for x in range(70):
        try :
            button = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@class='curiocity-load-more link link--underlined']")))
        except Exception as e:
            print(e)
        else :
            if button.is_displayed():
                driver.execute_script("arguments[0].click();", button)
                time.sleep(3)
page_source = driver.page_source

Message: 
Stacktrace:
0   chromedriver                        0x000000010ec18159 chromedriver + 5120345
1   chromedriver                        0x000000010eba5b13 chromedriver + 4651795
2   chromedriver                        0x000000010e795e68 chromedriver + 392808
3   chromedriver                        0x000000010e7cb181 chromedriver + 610689
4   chromedriver                        0x000000010e7cb341 chromedriver + 611137
5   chromedriver                        0x000000010e7fda74 chromedriver + 817780
6   chromedriver                        0x000000010e7e8b6d chromedriver + 732013
7   chromedriver                        0x000000010e7fb637 chromedriver + 808503
8   chromedriver                        0x000000010e7e8a33 chromedriver + 731699
9   chromedriver                        0x000000010e7be5dd chromedriver + 558557
10  chromedriver                        0x000000010e7bf4f5 chromedriver + 562421
11  chromedriver                        0x000000010ebd538d chromedriver + 4846477
12 

In [141]:
# taking the links for Vancouver
soup = BeautifulSoup(page_source, 'lxml')
cards = []
cards_selector = soup.find_all('article', class_='card')
for card_selector in cards_selector:
    header = card_selector.h3
    a = header.find('a', class_ ='card__link')
    link = a['href']
    title = a.text
    title = title.lstrip()
    food_and_drink_links[title] = link 

In [142]:
len(food_and_drink_links)

541

In [143]:
driver.get(cards_selector[5].h3.a["href"])
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [125]:
# Checking everything is working
card_1item = soup.find('main', class_="grid__item").findAll('h3')
TITLE = soup.find('div', class_="card__header") # This is the title of the card (is not especial to the ONE_CARD)
TITLE = TITLE.h1.text.strip('\n')
CATEGORY = soup.find('dl', class_="post__category").dd.a.text
tags = soup.find('div', class_="post__tags")
if tags == None:
    print("1")
#     TAGS_NAMES = ""
LINK = soup.find('figure', class_="wide").img["src"]
#     for tag in tags:
#         TAGS_NAMES += (tag.text + ", ")
#     TAGS_NAMES
DESCRIPTION = soup.find('div', class_="post-content grid__item")
DESCRIPTION = DESCRIPTION.findAll("p")
DESCRIPTION_FULL = ""
WHERE = ""
WHEN = ""
for p in DESCRIPTION:
    if 'Where:' in p.text :
        atts = p.text
        try :
            atts = atts.split("\n")
        except :
            print('Where and when not breakable')
        else :
            WHERE = atts[0][6:].lstrip(" ")
            WHEN = atts[1][5:].lstrip(" ")
    else :
        DESCRIPTION_FULL += (p.text + " ")
TITLE

"Vancouver is getting a NSFW dessert spot with 'willy' shaped waffles"

In [161]:
scrapped_vanc = []
to_scrap_vanc = []

In [162]:
# Scrapper of cards with just one item for Vancouver
naive_card_db_vanc = {'title':[], 'description':[], 'when':[], 'where':[], 'link':[], 'category':[], 'tags':[]}
naive_card_db_vanc = pd.DataFrame(naive_card_db_vanc)

for link in food_and_drink_links:
    driver.get(food_and_drink_links[link])
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    card_1item = soup.find('main', class_="grid__item").findAll('h3')
    if (len(card_1item) == 1) | (link in scrapped):
        TITLE = soup.find('div', class_="card__header") # This is the title of the card (is not especial to the ONE_CARD)
        TITLE = TITLE.h1.text.strip('\n')
        LINK = soup.find('figure', class_="wide").img["src"]
        CATEGORY = soup.find('dl', class_="post__category").dd.a.text
        tags = soup.find('div', class_="post__tags")
        TAGS_NAMES = ""
        if (tags != None):
            tags = tags.findAll('a')
            for tag in tags:
                TAGS_NAMES += (tag.text + ", ")
        DESCRIPTION = soup.find('div', class_="post-content grid__item")
        DESCRIPTION = DESCRIPTION.findAll("p")
        DESCRIPTION_FULL = ""
        WHERE = ""
        WHEN = ""
        for p in DESCRIPTION:
            if 'Where:' in p.text :
                atts = p.text
                try :
                    atts = atts.split("\n")
                except :
                    print('Where and when not breakable')
                else :
                    if (len(atts) >= 2):
                        WHERE = atts[0][6:].lstrip(" ")
                        WHEN = atts[1][5:].lstrip(" ")
                        row = {'title':TITLE, 'description':DESCRIPTION_FULL, 'when':WHEN, 'where':WHERE, 'link':LINK, 'category':CATEGORY}
                        naive_card_db_vanc = naive_card_db_vanc.append(row, ignore_index=True)
                    else :
                        row = {'title':TITLE, 'description':DESCRIPTION_FULL, 'when':"n/a", 'where':atts[0][6:], 'link':LINK, 'category':CATEGORY}
                        naive_card_db_vanc = naive_card_db_vanc.append(row, ignore_index=True)
                        print(atts)
            else :
                DESCRIPTION_FULL += (p.text + " ")
        print(TITLE)
        scrapped_vanc.append(link)
    else :
        to_scrap_vanc.append(link)
    time.sleep(1.5)

Gastown's wild new arcade bar opens this weekend & you're invited to the fun
You can get boozy juice boxes & all types of BBQ skewers at this Metro Vancouver spot
Richmond Night Market is the largest of its kind in North America & it's back this week
Vancouver is getting a NSFW dessert spot with 'willy' shaped waffles
A stylish new Italian restaurant is opening in Coal Harbour this spring
['Where: 1898 Main Street']
This hidden bar in Vancouver features all-day happy hour deals
Vancouver's Spot Prawn Festival is returning & it includes a brunch tasting menu
A retro-style neighbourhood bar & grill is coming to Mount Pleasant
A savoury Korean pancake house is coming to Vancouver
Vancouver's beloved 'PattyPass' comes back next month with 50% off burgers around town
A new series of beer and music fests is coming to Metro Vancouver
New Mexican restaurant to open in an iconic Vancouver spot this week
A Vancouver burger spot is back with a hidden bar attached
This could be your last chance to

In [164]:
naive_card_db_vanc.head()

,title,description,when,where,link,category,tags
0,Gastown's wild new arcade bar opens this weeke...,"GRETA YVR is just about ready to roll, and you...",: 50 W Cordova Street,"Opens Friday, April 29th",https://curiocity.com/wp-content/uploads/2022/...,Food & Drink,NaN
1,You can get boozy juice boxes & all types of B...,We know what you’re thinking — boozy juice box...,gram: @bbq.tasty,"180-7771 Westminster Highway, Richmond",https://curiocity.com/wp-content/uploads/2022/...,Food & Drink,NaN
2,Richmond Night Market is the largest of its ki...,"Get your forks ready, Vancouver. The Richmond ...",": 8351 River Road, Richmond","Friday, April 29th",https://curiocity.com/wp-content/uploads/2022/...,Food & Drink,NaN
3,Vancouver is getting a NSFW dessert spot with ...,"Great news, funky food enthusiasts. A raunchy ...",: 401 Industrial Avenue,"Friday, May 13th",https://curiocity.com/wp-content/uploads/2022/...,Food & Drink,NaN
4,A stylish new Italian restaurant is opening in...,"This spring, a new contemporary Italian restau...",: 1575 West Georgia,TBA,https://curiocity.com/wp-content/uploads/2022/...,Food & Drink,NaN


In [165]:
len(scrapped_vanc), len(scrapped)

(206, 193)

In [166]:
len(to_scrap), len(to_scrap_vanc)

(297, 335)

In [167]:
result = naive_card_db_vanc
result.to_json('~/Documents/INFO/food_drinks_vancouver_card_1.json', orient='table')
result.to_csv('~/Documents/INFO/food_drinks_vancouver_card_1.csv', index=True)

In [170]:
food_and_drinks_vanc = food_and_drink_links

### Seattle's card1 food_and_drinks

In [171]:
from selenium import webdriver
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome("/usr/local/bin/chromedriver", chrome_options=options)

<ipython-input-171-2c6674e248ce>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/local/bin/chromedriver", chrome_options=options)
<ipython-input-171-2c6674e248ce>:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome("/usr/local/bin/chromedriver", chrome_options=options)


In [172]:
food_and_drink_links_sea = dict()
print(food_and_drink_links_sea)

{}


In [173]:
# getting the html after clicks for seattles food card1
import sys

wait = WebDriverWait(driver, 5)
driver.get("https://curiocity.com/category/food-drink/?city=seattle")
for x in range(75):
        try :
            button = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@class='curiocity-load-more link link--underlined']")))
        except Exception as e:
            sys.exit(0)
        else :
            if button.is_displayed():
                driver.execute_script("arguments[0].click();", button)
                time.sleep(3)
page_source = driver.page_source

time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait
time limit for wait


In [174]:
# getting the links for seattles cards items
soup = BeautifulSoup(page_source, 'lxml')
cards = []
cards_selector = soup.find_all('article', class_='card')
for card_selector in cards_selector:
    header = card_selector.h3
    a = header.find('a', class_ ='card__link')
    link = a['href']
    title = a.text
    title = title.lstrip()
    food_and_drink_links_sea[title] = link 

In [175]:
len(food_and_drink_links_sea)

299

In [177]:
scrapped_sea = []
to_scrap_sea = []

In [178]:
# Scrapper of cards with just one item for Seattle
naive_card_db_sea = {'title':[], 'description':[], 'when':[], 'where':[], 'link':[], 'category':[], 'tags':[]}
naive_card_db_sea = pd.DataFrame(naive_card_db_sea)

for link in food_and_drink_links_sea:
    driver.get(food_and_drink_links_sea[link])
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    card_1item = soup.find('main', class_="grid__item").findAll('h3')
    if (len(card_1item) == 1) | (link in scrapped_sea):
        TITLE = soup.find('div', class_="card__header") # This is the title of the card (is not especial to the ONE_CARD)
        TITLE = TITLE.h1.text.strip('\n')
        LINK = soup.find('figure', class_="wide").img["src"]
        CATEGORY = soup.find('dl', class_="post__category").dd.a.text
        tags = soup.find('div', class_="post__tags")
        TAGS_NAMES = ""
        if (tags != None):
            tags = tags.findAll('a')
            for tag in tags:
                TAGS_NAMES += (tag.text + ", ")
        DESCRIPTION = soup.find('div', class_="post-content grid__item")
        DESCRIPTION = DESCRIPTION.findAll("p")
        DESCRIPTION_FULL = ""
        WHERE = ""
        WHEN = ""
        for p in DESCRIPTION:
            if 'Where:' in p.text :
                atts = p.text
                try :
                    atts = atts.split("\n")
                except :
                    print('Where and when not breakable')
                else :
                    if (len(atts) >= 2):
                        WHERE = atts[0][6:].lstrip(" ")
                        WHEN = atts[1][5:].lstrip(" ")
                        row = {'title':TITLE, 'description':DESCRIPTION_FULL, 'when':WHEN, 'where':WHERE, 'link':LINK, 'category':CATEGORY}
                        naive_card_db_sea = naive_card_db_sea.append(row, ignore_index=True)
                    else :
                        row = {'title':TITLE, 'description':DESCRIPTION_FULL, 'when':"n/a", 'where':atts[0][6:], 'link':LINK, 'category':CATEGORY}
                        naive_card_db_sea = naive_card_db_sea.append(row, ignore_index=True)
                        print(atts)
            else :
                DESCRIPTION_FULL += (p.text + " ")
        print(TITLE)
        scrapped_sea.append(link)
    else :
        to_scrap_sea.append(link)
    time.sleep(3)

Here's the tea: The first ever Seattle Boba Fest is happening in the U District this weekend
This Capitol Hill wine shop offers a monthly natural wine club subscription
The chef behind Bainbridge Island's Hitchcock is opening a new resto next month
Lucky Envelope Brewing is hosting a series of tasty pop ups this May & you don't want to miss out
Money Frog is Capitol Hill's newest spot for pan-Asian eats & it opens tomorrow
Foodie Fest is bringing together 13 of Seattle's most popular pop ups this April
Seattle Restaurant Week is back and on now with over 200 participants
The Fairmont Olympic's stunning new restaurant officially opens today
Seattle's newest Korean restaurant just opened in Ballard
Final Call: This is your last month ever to grab a brew at Hale's Ales
Wolf and Woman & chef Tiffany Ran are collabing for an unforgettable pop up this April
Hi Life is Bainbridge Island's newest poke spot & is worth the ferry ride
Kamino Sushi and More is Seattle's only Burmese resto & you ca

In [182]:
naive_card_db_sea.shape

(116, 7)

In [183]:
result = naive_card_db_sea
result.to_json('~/Documents/INFO/food_drinks_seattle_card_1.json', orient='table')
result.to_csv('~/Documents/INFO/food_drinks_seattle_card_1.csv', index=True)

### Calgary card1 food and drinks

In [184]:
from selenium import webdriver
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome("/usr/local/bin/chromedriver", chrome_options=options)

<ipython-input-184-2c6674e248ce>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/local/bin/chromedriver", chrome_options=options)
<ipython-input-184-2c6674e248ce>:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome("/usr/local/bin/chromedriver", chrome_options=options)


In [185]:
food_and_drink_links_calg = dict()
print(food_and_drink_links_calg)

{}


In [187]:
# getting the html after clicks for calgary food card1
import sys

wait = WebDriverWait(driver, 5)
driver.get("https://curiocity.com/category/food-drink/?city=calgary")
for x in range(85):
        try :
            button = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@class='curiocity-load-more link link--underlined']")))
        except Exception as e:
            print("error")
        else :
            if button.is_displayed():
                driver.execute_script("arguments[0].click();", button)
                time.sleep(3)
page_source = driver.page_source

error
error
error
error
error
error


In [188]:
# getting the links for seattles cards items
soup = BeautifulSoup(page_source, 'lxml')
cards_selector = soup.find_all('article', class_='card')
for card_selector in cards_selector:
    header = card_selector.h3
    a = header.find('a', class_ ='card__link')
    link = a['href']
    title = a.text
    title = title.lstrip()
    food_and_drink_links_calg[title] = link

In [189]:
len(food_and_drink_links_calg)

625

In [190]:
scrapped_calg = []
to_scrap_calg = []

In [191]:
# Scrapper of cards with just one item for Calgary
naive_card_db_calg = {'title':[], 'description':[], 'when':[], 'where':[], 'link':[], 'category':[], 'tags':[]}
naive_card_db_calg = pd.DataFrame(naive_card_db_calg)

for link in food_and_drink_links_calg:
    driver.get(food_and_drink_links_calg[link])
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    card_1item = soup.find('main', class_="grid__item").findAll('h3')
    if (len(card_1item) == 1) | (link in scrapped_calg):
        TITLE = soup.find('div', class_="card__header") # This is the title of the card (is not especial to the ONE_CARD)
        TITLE = TITLE.h1.text.strip('\n')
        LINK = soup.find('figure', class_="wide").img["src"]
        CATEGORY = soup.find('dl', class_="post__category").dd.a.text
        tags = soup.find('div', class_="post__tags")
        TAGS_NAMES = ""
        if (tags != None):
            tags = tags.findAll('a')
            for tag in tags:
                TAGS_NAMES += (tag.text + ", ")
        DESCRIPTION = soup.find('div', class_="post-content grid__item")
        DESCRIPTION = DESCRIPTION.findAll("p")
        DESCRIPTION_FULL = ""
        WHERE = ""
        WHEN = ""
        for p in DESCRIPTION:
            if 'Where:' in p.text :
                atts = p.text
                try :
                    atts = atts.split("\n")
                except :
                    print('Where and when not breakable')
                else :
                    if (len(atts) >= 2):
                        WHERE = atts[0][6:].lstrip(" ")
                        WHEN = atts[1][5:].lstrip(" ")
                        row = {'title':TITLE, 'description':DESCRIPTION_FULL, 'when':WHEN, 'where':WHERE, 'link':LINK, 'category':CATEGORY}
                        naive_card_db_calg = naive_card_db_calg.append(row, ignore_index=True)
                    else :
                        row = {'title':TITLE, 'description':DESCRIPTION_FULL, 'when':"n/a", 'where':atts[0][6:], 'link':LINK, 'category':CATEGORY}
                        naive_card_db_calg = naive_card_db_calg.append(row, ignore_index=True)
                        print(atts)
            else :
                DESCRIPTION_FULL += (p.text + " ")
        print(TITLE)
        scrapped_calg.append(link)
    else :
        to_scrap_calg.append(link)
    time.sleep(3)

A colorful new breakfast spot just opened in Calgary with uniquely sweet sips & big brunch trays
This popular Calgary coffee shop known for their cronuts just opened a new location downtown
A chic new cocktail bar for 'non-conformists' just opened on 17th Avenue in Calgary
Taste of Calgary is returning for its 25th anniversary & here's what to expect
CRAFT is launching a new concept in Calgary & it opens next month
This adorable mobile brewery in Calgary is like an ice cream truck for adults
Get $9 3/4 'potions' at Calgary's magical Harry Potter-themed bar during happy hour
This popular cocktail bar in Calgary is hosting a tasty tapas popup this weekend
A cafe & flower shop hybrid just opened in Calgary with colorful lattes, desserts & brunch
['Where: 4407 1 St SE, Calgary, AB T2G 2L2']
Calgary is the only city where you can grab a beer from the 'Canadian Brewery of the Year'
['Where: 3851 Manchester Road SE, Calgary']
Calgary spot known for $4 tacos & mini golf course reopens after 3-

In [192]:
naive_card_db_calg

,title,description,when,where,link,category,tags
0,A colorful new breakfast spot just opened in C...,"One can never have too many clothes, too many ...",gram: @brouhaha_breakfast,609 14th NW. Calgary AB,https://curiocity.com/wp-content/uploads/2022/...,Food & Drink,NaN
1,This popular Calgary coffee shop known for the...,The only thing better than a grand opening – i...,": 308 4 Ave SW, Calgary","Tuesday, April 26th",https://curiocity.com/wp-content/uploads/2022/...,Food & Drink,NaN
2,A chic new cocktail bar for 'non-conformists' ...,When it comes to cocktail bars we can’t get en...,gram: @nueyyc,"608 17th Ave SW, Calgary",https://curiocity.com/wp-content/uploads/2022/...,Food & Drink,NaN
3,Taste of Calgary is returning for its 25th ann...,"Forks at the ready, folks, because Taste of Ca...",": 311, 8 Street SW","August 4-7, 2022",https://curiocity.com/wp-content/uploads/2022/...,Food & Drink,NaN
4,CRAFT is launching a new concept in Calgary & ...,"Good food, good drinks, and good trouble? Whil...",": 224 12 Ave SW #110, Calgary,","Monday, May 9th",https://curiocity.com/wp-content/uploads/2022/...,Food & Drink,NaN
...,...,...,...,...,...,...,...
288,"The brand-new, reimagined CRAFT Beer Market is...",Who’s been patiently waiting for CRAFT Beer Ma...,n/a,345 10 Ave SW,https://curiocity.com/wp-content/uploads/2021/...,Food & Drink,NaN
289,Pizza 73 just launched a sneaky healthy caulif...,Pizza? But not nutritional trash? We’re all ea...,n/a,Find your nearest location here,https://curiocity.com/wp-content/uploads/2021/...,Food & Drink,NaN
290,This 1940s diner is one of Calgary's most icon...,"Throwback diner time? Yeah, we thought so too....",n/a,802 12 St SE,https://curiocity.com/wp-content/uploads/2021/...,Food & Drink,NaN
291,An authentic NY-style pizza joint is opening i...,"When it comes to pizza places in Calgary, we c...",: 390 Northmount Drive NW,Opening Soon,https://curiocity.com/wp-content/uploads/2021/...,Food & Drink,NaN


In [194]:
len(to_scrap_calg), 

328

In [196]:
food_and_drink_links_calg[to_scrap_calg[0]]

'https://curiocity.com/themed-bars-calgary/'

In [195]:
result = naive_card_db_calg
result.to_json('~/Documents/INFO/food_drinks_calgary_card_1.json', orient='table')
result.to_csv('~/Documents/INFO/food_drinks_calgary_card_1.csv', index=True)

# Edmonton food and drinks card1

In [197]:
from selenium import webdriver
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome("/usr/local/bin/chromedriver", chrome_options=options)

<ipython-input-197-2c6674e248ce>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/local/bin/chromedriver", chrome_options=options)
<ipython-input-197-2c6674e248ce>:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome("/usr/local/bin/chromedriver", chrome_options=options)


In [198]:
food_and_drink_links_edm = dict()
print(food_and_drink_links_edm)

{}


In [199]:
# getting the html after clicks for calgary food card1
import sys

wait = WebDriverWait(driver, 5)
driver.get("https://curiocity.com/category/food-drink/?city=edmonton")
for x in range(85):
        try :
            button = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@class='curiocity-load-more link link--underlined']")))
        except Exception as e:
            print("error")
        else :
            if button.is_displayed():
                driver.execute_script("arguments[0].click();", button)
                time.sleep(3)
page_source = driver.page_source

error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error


In [200]:
# getting the links for seattles cards items
soup = BeautifulSoup(page_source, 'lxml')
cards_selector = soup.find_all('article', class_='card')
for card_selector in cards_selector:
    header = card_selector.h3
    a = header.find('a', class_ ='card__link')
    link = a['href']
    title = a.text
    title = title.lstrip()
    food_and_drink_links_edm[title] = link

In [201]:
len(food_and_drink_links_edm)

293

In [202]:
scrapped_edm = []
to_scrap_edm = []

In [288]:
# Scrapper of cards with just one item for Edmonton
naive_card_db_edm = {'title':[], 'description':[], 'when':[], 'where':[], 'link':[], 'category':[], 'tags':[]}
naive_card_db_edm = pd.DataFrame(naive_card_db_edm)

for link in scrapped_edm:
    driver.get(food_and_drink_links_edm[link])
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    card_1item = soup.find('main', class_="grid__item").findAll('h3')
    if (len(card_1item) == 1) | (link in scrapped_edm):
        TITLE = soup.find('div', class_="card__header") # This is the title of the card (is not especial to the ONE_CARD)
        TITLE = TITLE.h1.text.strip('\n')
        LINK = soup.find('figure', class_="wide").img["src"]
        CATEGORY = soup.find('dl', class_="post__category").dd.a.text
        tags = soup.find('div', class_="post__tags")
        TAGS_NAMES = ""
        if (tags != None):
            tags = tags.findAll('a')
            for tag in tags:
                TAGS_NAMES += (tag.text + ", ")
        DESCRIPTION = soup.find('div', class_="post-content grid__item")
        DESCRIPTION = DESCRIPTION.findAll("p")
        DESCRIPTION_FULL = ""
        WHERE = ""
        WHEN = ""
        for p in DESCRIPTION:
            if 'Where:' in p.text :
                atts = p.text
                try :
                    atts = atts.split("\n")
                except :
                    print('Where and when not breakable')
                else :
                    if (len(atts) >= 2):
                        WHERE = atts[0][6:].lstrip(" ")
                        WHEN = atts[1][5:].lstrip(" ")
                        row = {'title':TITLE, 'description':DESCRIPTION_FULL, 'when':WHEN, 'where':WHERE, 'link':LINK, 'category':CATEGORY, 'tags':TAGS_NAMES}
                        naive_card_db_edm = naive_card_db_edm.append(row, ignore_index=True)
                    else :
                        row = {'title':TITLE, 'description':DESCRIPTION_FULL, 'when':"n/a", 'where':atts[0][6:], 'link':LINK, 'category':CATEGORY, 'tags':TAGS_NAMES}
                        naive_card_db_edm = naive_card_db_edm.append(row, ignore_index=True)
                        print(atts)
            else :
                DESCRIPTION_FULL += (p.text + " ")
        print(TITLE)
    time.sleep(2)

['Where: 14229 23 Ave NW']
One Edmonton restaurant just made a new list of Canada's best brunch spots
['Where: Whyte Avenue and 105th Street']
Whyte Ave’s highly-awaited bar, Rally Social Room, opens tonight & here’s what to expect
This new Edmonton pastry shop serves adorable teddy bear shaped desserts & fish cones
Edmonton's chic new cocktail bar serves pots of beer, all-day dumplings & boozy bubble tea
A new Latin American restaurant has opened in Edmonton
Boozy bike tours are coming back to Edmonton this month
Dickey's, the world's largest BBQ Concept, to open first Alberta location in Edmonton
A brand new coffee shop just opened its doors in Edmonton
Sip tea in a castle tower at this elegant 'Bridgerton-like' experience in Banff
What to know about the return of Downtown Dining Week in Edmonton
This popular café is hosting a tasty ramen popup in Edmonton this weekend
Indoor dining has returned to this iconic Edmonton bakery
['Where: 10363 104 St NW, Edmonton']
ICE District’s new sp

In [314]:
naive_card_db_edm.link[33]

'https://curiocity.com/wp-content/uploads/2021/10/Untitled-design-86-1255x600.png'

In [290]:
result = naive_card_db_edm
result.to_json('~/Documents/INFO/Capstone22/recsys/food_drinks_edmonton_card_1.json', orient='table')
result.to_csv('~/Documents/INFO/Capstone22/recsys/food_drinks_edmonton_card_1.csv', index=True)

# The second step CARD2
2. Now we have to get all the cardsX that are the ones that have more than 1 item in the links

In [206]:
for title in to_scrap_edm:
    print(food_and_drink_links_edm[title])

https://curiocity.com/new-restaurants-bars-edmonton-spring-2022/
https://curiocity.com/10-edmonton-eats-that-are-perfect-for-your-4-20-munchies/
https://curiocity.com/pint-free-wings-every-wednesday-this-month/
https://curiocity.com/10-secret-menu-items-you-can-try-at-canadas-favourite-fast-food-joints/
https://curiocity.com/quesada-burritos/
https://curiocity.com/good-earth-indigo-expansion/
https://curiocity.com/mcdonalds-mcgriddles-canada/
https://curiocity.com/edmonton-filipino-restaurant-month/
https://curiocity.com/best-hidden-bars-speakeasies-edmonton/
https://curiocity.com/odd-burger-western-canada-expansion/
https://curiocity.com/this-resto-has-weekday-happy-hours-starting-at-5/
https://curiocity.com/most-scenic-restaurants-alberta/
https://curiocity.com/mcbroken-mcdonalds-ice-cream-machines/
https://curiocity.com/dominos-pizza-deal-canada-march-7th/
https://curiocity.com/pebbles-cereals-canada/
https://curiocity.com/filipino-restaurant-month-canada/
https://curiocity.com/win-

In [315]:
driver.get("https://curiocity.com/10-edmonton-eats-that-are-perfect-for-your-4-20-munchies/")
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [322]:
naive_card_db_ = {'title':[], 'description':[], 'when':[], 'where':[], 'link':[], 'category':[], 'tags':[]}
naive_card_db_ = pd.DataFrame(naive_card_db_)

for title in to_scrap_edm:
    driver.get(food_and_drink_links_edm[title])
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    div_grid = soup.find('div',class_='post-content grid__item')
    card_1item = soup.find('main', class_="grid__item").findAll('h3')
    headers = div_grid.findAll('h3')
    if (len(card_1item) > 1):
        CATEGORY = soup.find('dl', class_="post__category").dd.a.text
        TAGS_NAMES = ""
        tags = soup.find('div', class_="post__tags")
        try :
            tags = tags.findAll('a')
        except :
            print("error")
        else :
            if tags == None:
                print("1")
            else : 
                for tag in tags:
                    TAGS_NAMES += (tag.text + ", ")
        for header in headers:
            LINK = ""
            TITLE = header.text
            header_helper = header
            DESCRIPTION = ""
            WHERE = ""
            WHEN = ""
            while header_helper.next_element.name != 'h3':
                if (header_helper.name == "iframe"):
                    try :
                        link_creator = header_helper["class"]
                    except Exception as e :
                        print("error")
                    else :
                        if "instagram-media" in link_creator:
                            LINK = header_helper["src"]
                header_helper = header_helper.next_element
                if header_helper.name == "p":
                    if 'Where:' in header_helper.text :
                        atts = header_helper.text
                        try :
                            atts = atts.split("\n")
                        except :
                            print('Where and when not breakable')
                        else :
                            if (len(atts) >= 2):
                                WHERE = atts[0][6:].lstrip(" ")
                                WHEN = atts[1][5:].lstrip(" ")
                            else :
                                WHERE = atts[0][6:]
                    else :
                        DESCRIPTION += (header_helper.text + " ")
            #END Of while loop to check for h3
            row = {'title':TITLE, 'description':DESCRIPTION, 'when':WHEN, 'where':WHERE, 'link':LINK, 'category':CATEGORY, 'tags':TAGS_NAMES}
            naive_card_db_ = naive_card_db_.append(row, ignore_index=True)
    time.sleep(2)

In [326]:
naive_card_db_.link[5]

'https://www.instagram.com/p/CcbHCMHvleD/embed/captioned/?cr=1&v=14&wp=1516&rd=https%3A%2F%2Fcuriocity.com&rp=%2F10-edmonton-eats-that-are-perfect-for-your-4-20-munchies%2F#%7B%22ci%22%3A1%2C%22os%22%3A1699.5%2C%22ls%22%3A1260.3999999761581%2C%22le%22%3A1465.3999999761581%7D'

In [285]:
naive_card_db_edm

,title,description,when,where,link,category,tags
0,NOW OPEN,We might as well kick the list off with the c...,,10455 82 Avenue NW,,Food & Drink,"Food & Drink,"
1,OPENING SOON,"Yeah, we’re still holding out hope that the i...",: 10403 83 Avenue NW,Slated to open in 2022,,Food & Drink,"Food & Drink,"
2,K CORN DOGS FROM GANGNAM STREET FOOD,When struggling to fend off the munchies look ...,,,,Food & Drink,"Food & Drink, Restaurants,"
3,FRIED CHICKEN FROM NORTHERN CHICKEN,‘Cause nothing can cure a stoner craving quite...,,,,Food & Drink,"Food & Drink, Restaurants,"
4,THEY CALL ME BRUCE FROM HIGH DOUGH,Whoever said pineapples aren’t for pizzas had...,,,,Food & Drink,"Food & Drink, Restaurants,"
...,...,...,...,...,...,...,...
279,SMOKEHOUSE BBQ,Smokehouse BBQ is like a teeny Texas right he...,,15960 109 Ave NW,,Food & Drink,
280,SMOKE & ASH,It’s certainly not the cutest place on the li...,,"106 Fulton Drive, Acheson",,Food & Drink,
281,LEE HOUSE KOREAN RESTAURANT,"Korean BBQ is a way of life, people. Ribs, bu...",,10704 97 St NW,,Food & Drink,
282,PAMPA BRAZILIAN STEAKHOUSE,It might not be a traditional BBQ joint in th...,,9929 109 St NW,,Food & Drink,


2. Calgary

In [292]:
# SCRAP CALGARY CARDS

naive_card_db_calg = {'title':[], 'description':[], 'when':[], 'where':[], 'link':[], 'category':[], 'tags':[]}
naive_card_db_calg = pd.DataFrame(naive_card_db_calg)

for title in to_scrap_calg:
    driver.get(food_and_drink_links_calg[title])
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    div_grid = soup.find('div',class_='post-content grid__item')
    card_1item = soup.find('main', class_="grid__item").findAll('h3')
    headers = div_grid.findAll('h3')
    if (len(card_1item) > 1):
        CATEGORY = soup.find('dl', class_="post__category").dd.a.text
        TAGS_NAMES = ""
        tags = soup.find('div', class_="post__tags")
        try :
            tags = tags.findAll('a')
        except :
            print("error")
        else :
            if tags == None:
                print("1")
            else : 
                for tag in tags:
                    TAGS_NAMES += (tag.text + ", ")
        for header in headers:
            LINK = ""
            TITLE = header.text
            header_helper = header
            DESCRIPTION = ""
            WHERE = ""
            WHEN = ""
            while header_helper.next_element.name != 'h3':
                if (header_helper.name == "iframe"):
                    try :
                        link_creator = header_helper
                    except Exception as e :
                        print("error")
                    else :
                        ("non instagram")
                header_helper = header_helper.next_element
                if header_helper.name == "p":
                    if 'Where:' in header_helper.text :
                        atts = header_helper.text
                        try :
                            atts = atts.split("\n")
                        except :
                            print('Where and when not breakable')
                        else :
                            if (len(atts) >= 2):
                                WHERE = atts[0][6:].lstrip(" ")
                                WHEN = atts[1][5:].lstrip(" ")
                            else :
                                WHERE = atts[0][6:]
                    else :
                        DESCRIPTION += (header_helper.text + " ")
            #END Of while loop to check for h3
            print(TITLE)
            row = {'title':TITLE, 'description':DESCRIPTION, 'when':WHEN, 'where':WHERE, 'link':LINK, 'category':CATEGORY, 'tags':TAGS_NAMES}
            naive_card_db_calg = naive_card_db_calg.append(row, ignore_index=True)
    time.sleep(2)

error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error


In [293]:
naive_card_db_calg

,title,description,when,where,link,category,tags
0,SHELTER,A spot that we can only describe as embodying...,gram: @shelter.yyc,"1210 1 St SW, Calgary, AB",,Food & Drink,"Adventure, Attractions, Food & Drink,"
1,RICARDO’S HIDEAWAY,Need a vacation? A trip to Ricardo’s Hideaway...,gram: @ricardosyyc,"1530 5 St SW, Calgary, AB",,Food & Drink,"Adventure, Attractions, Food & Drink,"
2,THE POTION ROOM,"At Calgary’s only Harry Potter-themed bar, Th...",gram: @the.potionroom,"1211 9 Ave SE, Calgary, AB",,Food & Drink,"Adventure, Attractions, Food & Drink,"
3,BETTY LOU’S LIBRARY,"Nothing is cozier than a dark room, velour ch...",gram: @bettylouslibary,"The Devenish Building, 908 17th Avenue SW, Ca...",,Food & Drink,"Adventure, Attractions, Food & Drink,"
4,PAPER LANTERN,"Located in a Chinatown basement, you’ll find ...",gram: @paperlanteryyc,"Basement – 115 – 2 Avenue SE, Calgary",,Food & Drink,"Adventure, Attractions, Food & Drink,"
...,...,...,...,...,...,...,...
1013,Bridgette Bar,"Date night, after-work drinks, a girls’ (or g...",,739 10 Ave SW,,Food & Drink,
1014,The Wednesday Room,Walking into The Wednesday Room is like walki...,,739 10 Ave SW,,Food & Drink,
1015,Mister Chen’s,"Besides their delicious and innovative menu, ...",,2004 4 St SW,,Food & Drink,
1016,Bread and Circus,"Inspired by Rome, Bread and Circus is a fun, ...",,,,Food & Drink,


In [294]:
result = naive_card_db_calg
result.to_json('~/Documents/INFO/Capstone22/recsys/food_drinks_calgary_card_X.json', orient='table')
result.to_csv('~/Documents/INFO/Capstone22/recsys/food_drinks_calgary_card_X.csv', index=True)

# Vancouver card2 food_drinks

In [295]:
len(to_scrap_vanc)

335

In [299]:
# SCRAP VANCOUVER CARDS

naive_card_db_vanc = {'title':[], 'description':[], 'when':[], 'where':[], 'link':[], 'category':[], 'tags':[]}
naive_card_db_vanc = pd.DataFrame(naive_card_db_vanc)

for title in to_scrap_vanc:
    driver.get(food_and_drink_links[title])
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    div_grid = soup.find('div',class_='post-content grid__item')
    card_1item = soup.find('main', class_="grid__item").findAll('h3')
    headers = div_grid.findAll('h3')
    if (len(card_1item) > 1):
        CATEGORY = soup.find('dl', class_="post__category").dd.a.text
        TAGS_NAMES = ""
        tags = soup.find('div', class_="post__tags")
        try :
            tags = tags.findAll('a')
        except :
            print("error")
        else :
            if tags == None:
                print("1")
            else : 
                for tag in tags:
                    TAGS_NAMES += (tag.text + ", ")
        for header in headers:
            LINK = ""
            TITLE = header.text
            header_helper = header
            DESCRIPTION = ""
            WHERE = ""
            WHEN = ""
            while header_helper.next_element.name != 'h3':
                if (header_helper.name == "iframe"):
                    try :
                        link_creator = header_helper
                    except Exception as e :
                        print("error")
                    else :
                        ("non instagram")
                header_helper = header_helper.next_element
                if header_helper.name == "p":
                    if 'Where:' in header_helper.text :
                        atts = header_helper.text
                        try :
                            atts = atts.split("\n")
                        except :
                            print('Where and when not breakable')
                        else :
                            if (len(atts) >= 2):
                                WHERE = atts[0][6:].lstrip(" ")
                                WHEN = atts[1][5:].lstrip(" ")
                            else :
                                WHERE = atts[0][6:]
                    else :
                        DESCRIPTION += (header_helper.text + " ")
            #END Of while loop to check for h3
            print(TITLE)
            row = {'title':TITLE, 'description':DESCRIPTION, 'when':WHEN, 'where':WHERE, 'link':LINK, 'category':CATEGORY, 'tags':TAGS_NAMES}
            naive_card_db_vanc = naive_card_db_vanc.append(row, ignore_index=True)
    time.sleep(2)

error
THE RUNDOWN ON DEWAR’S SCOTCH-WHISKY
THE DEWAR’S SCOTCH-WHISKY PROCESS
THE SUMMERY SIPS
THE LOW DOWN ON SULTAN
THE INTERVIEW
THE TAKEAWAY
Neverland Tea Salon
Tableau Bar Bistro
Earls Kitchen + Bar
Botanist
LIFT Bar Grill View
Top of Vancouver Revolving Restaurant
Homer Street Cafe and Bar
Miku Restaurant
West Oak Restaurant
error
THE RUNDOWN ON DEWAR’S SCOTCH-WHISKY
THE DEWAR’S SCOTCH-WHISKY PROCESS
THE INTERVIEW
THE TAKEAWAY
error
DAIRY QUEEN – BANANA SPLIT BLIZZARD
DAIRY QUEEN – MOCHA COFFEE BLIZZARD
CHIPOTLE – NACHOS
MCDONALD’S – THE MCGANGBANG/THE MCNASTY
MCDONALD’S – FULLY LOADED MCFLURRY
MCDONALD’S – THE DOUBLE MCCHICKEN
STARBUCKS – “CINNAMON TOAST CRUNCH” LATTE
STARBUCKS – OATMEAL COOKIE LATTE
STARBUCKS – “TEDDY GRAHAM” LATTE
FIVE GUYS – “IN-N-OUT ANIMAL STYLE” BURGER
Chocolates
Brunch Treats
Other Options
Vancouver
Surrey
Richmond
New Westminster
Delta
Now Open
Opening Soon
Cantina 189
Chupito
Laowai
Key Party
The Narrow Lounge
Guilt & Co.
Hidden Lounge
The Elk Room
The S

In [300]:
naive_card_db_vanc

,title,description,when,where,link,category,tags
0,THE RUNDOWN ON DEWAR’S SCOTCH-WHISKY,"In fact, Dewar’s has been leading the scotch-w...",,,,Food & Drink,
1,THE DEWAR’S SCOTCH-WHISKY PROCESS,Every whisky in the Cask Series starts with a ...,,,,Food & Drink,
2,THE SUMMERY SIPS,"Now back to summertime drinks, one of Dewar’s ...",,,,Food & Drink,
3,THE LOW DOWN ON SULTAN,"As a Torontonian for twelve years, Sultan was...",,,,Food & Drink,
4,THE INTERVIEW,Q: What’s your favourite way to drink Dewar’s?...,,,,Food & Drink,
...,...,...,...,...,...,...,...
1002,Bon’s Off Broadway,If you’re ready to wear the smallest beanie i...,: 2451 Nanaimo St,"Monday to Friday, 7AM-8PM, and Saturday/Sunday...",,Food & Drink,
1003,Mr. Red Cafe,Tied with Ba Le for best cheap Vietnamese cui...,: 2680 W Broadway and 2131 E Hastings St,"Everyday, 11AM-9PM",,Food & Drink,
1004,Bestie,With probably the most specific cuisine on th...,: 105 E Pender St,"Sunday to Thursday, 11:30AM-10PM and Friday/Sa...",,Food & Drink,
1005,Kam Wai Dim Sum,"When you’re wholesaling to T&T, it’s pretty s...",: 249 E Pender St,"Everyday, 9AM-6PM",,Food & Drink,


In [301]:
result = naive_card_db_vanc
result.to_json('~/Documents/INFO/Capstone22/recsys/food_drinks_vancouver_card_X.json', orient='table')
result.to_csv('~/Documents/INFO/Capstone22/recsys/food_drinks_vancouver_card_X.csv', index=True)

# Calgary card2 food_drink

In [305]:
# SCRAP Calgary CARDS

naive_card_db_calg = {'title':[], 'description':[], 'when':[], 'where':[], 'link':[], 'category':[], 'tags':[]}
naive_card_db_calg = pd.DataFrame(naive_card_db_calg)

for title in to_scrap_calg:
    driver.get(food_and_drink_links_calg[title])
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    div_grid = soup.find('div',class_='post-content grid__item')
    card_1item = soup.find('main', class_="grid__item").findAll('h3')
    headers = div_grid.findAll('h3')
    if (len(card_1item) > 1):
        CATEGORY = soup.find('dl', class_="post__category").dd.a.text
        TAGS_NAMES = ""
        tags = soup.find('div', class_="post__tags")
        try :
            tags = tags.findAll('a')
        except :
            print("error")
        else :
            if tags == None:
                print("1")
            else : 
                for tag in tags:
                    TAGS_NAMES += (tag.text + ", ")
        for header in headers:
            LINK = ""
            TITLE = header.text
            header_helper = header
            DESCRIPTION = ""
            WHERE = ""
            WHEN = ""
            while header_helper.next_element.name != 'h3':
                if (header_helper.name == "iframe"):
                    try :
                        link_creator = header_helper
                    except Exception as e :
                        print("error")
                    else :
                        ("non instagram")
                header_helper = header_helper.next_element
                if header_helper.name == "p":
                    if 'Where:' in header_helper.text :
                        atts = header_helper.text
                        try :
                            atts = atts.split("\n")
                        except :
                            print('Where and when not breakable')
                        else :
                            if (len(atts) >= 2):
                                WHERE = atts[0][6:].lstrip(" ")
                                WHEN = atts[1][5:].lstrip(" ")
                            else :
                                WHERE = atts[0][6:]
                    else :
                        DESCRIPTION += (header_helper.text + " ")
            #END Of while loop to check for h3
            row = {'title':TITLE, 'description':DESCRIPTION, 'when':WHEN, 'where':WHERE, 'link':LINK, 'category':CATEGORY, 'tags':TAGS_NAMES}
            naive_card_db_calg = naive_card_db_calg.append(row, ignore_index=True)
    time.sleep(2)

error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error


In [306]:
naive_card_db_calg

,title,description,when,where,link,category,tags
0,SHELTER,A spot that we can only describe as embodying...,gram: @shelter.yyc,"1210 1 St SW, Calgary, AB",,Food & Drink,"Adventure, Attractions, Food & Drink,"
1,RICARDO’S HIDEAWAY,Need a vacation? A trip to Ricardo’s Hideaway...,gram: @ricardosyyc,"1530 5 St SW, Calgary, AB",,Food & Drink,"Adventure, Attractions, Food & Drink,"
2,THE POTION ROOM,"At Calgary’s only Harry Potter-themed bar, Th...",gram: @the.potionroom,"1211 9 Ave SE, Calgary, AB",,Food & Drink,"Adventure, Attractions, Food & Drink,"
3,BETTY LOU’S LIBRARY,"Nothing is cozier than a dark room, velour ch...",gram: @bettylouslibary,"The Devenish Building, 908 17th Avenue SW, Ca...",,Food & Drink,"Adventure, Attractions, Food & Drink,"
4,PAPER LANTERN,"Located in a Chinatown basement, you’ll find ...",gram: @paperlanteryyc,"Basement – 115 – 2 Avenue SE, Calgary",,Food & Drink,"Adventure, Attractions, Food & Drink,"
...,...,...,...,...,...,...,...
1013,Bridgette Bar,"Date night, after-work drinks, a girls’ (or g...",,739 10 Ave SW,,Food & Drink,
1014,The Wednesday Room,Walking into The Wednesday Room is like walki...,,739 10 Ave SW,,Food & Drink,
1015,Mister Chen’s,"Besides their delicious and innovative menu, ...",,2004 4 St SW,,Food & Drink,
1016,Bread and Circus,"Inspired by Rome, Bread and Circus is a fun, ...",,,,Food & Drink,


In [307]:
result = naive_card_db_calg
result.to_json('~/Documents/INFO/Capstone22/recsys/food_drinks_calgary_card_X.json', orient='table')
result.to_csv('~/Documents/INFO/Capstone22/recsys/food_drinks_calgary_card_X.csv', index=True)

## Seattle CARDS2

In [327]:
naive_card_db_sea = {'title':[], 'description':[], 'when':[], 'where':[], 'link':[], 'category':[], 'tags':[]}
naive_card_db_sea = pd.DataFrame(naive_card_db_sea)

for title in to_scrap_sea:
    driver.get(food_and_drink_links_sea[title])
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    div_grid = soup.find('div',class_='post-content grid__item')
    card_1item = soup.find('main', class_="grid__item").findAll('h3')
    headers = div_grid.findAll('h3')
    if (len(card_1item) > 1):
        CATEGORY = soup.find('dl', class_="post__category").dd.a.text
        TAGS_NAMES = ""
        tags = soup.find('div', class_="post__tags")
        try :
            tags = tags.findAll('a')
        except :
            print("error")
        else :
            if tags == None:
                print("1")
            else : 
                for tag in tags:
                    TAGS_NAMES += (tag.text + ", ")
        for header in headers:
            LINK = ""
            TITLE = header.text
            header_helper = header
            DESCRIPTION = ""
            WHERE = ""
            WHEN = ""
            while header_helper.next_element.name != 'h3':
                if (header_helper.name == "iframe"):
                    try :
                        link_creator = header_helper["class"]
                    except Exception as e :
                        print("error")
                    else :
                        if "instagram-media" in link_creator:
                            LINK = header_helper["src"]
                header_helper = header_helper.next_element
                if header_helper.name == "p":
                    if 'Where:' in header_helper.text :
                        atts = header_helper.text
                        try :
                            atts = atts.split("\n")
                        except :
                            print('Where and when not breakable')
                        else :
                            if (len(atts) >= 2):
                                WHERE = atts[0][6:].lstrip(" ")
                                WHEN = atts[1][5:].lstrip(" ")
                            else :
                                WHERE = atts[0][6:]
                    else :
                        DESCRIPTION += (header_helper.text + " ")
            #END Of while loop to check for h3
            row = {'title':TITLE, 'description':DESCRIPTION, 'when':WHEN, 'where':WHERE, 'link':LINK, 'category':CATEGORY, 'tags':TAGS_NAMES}
            naive_card_db_sea = naive_card_db_sea.append(row, ignore_index=True)
    time.sleep(2)

error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error


In [328]:
naive_card_db_sea

,title,description,when,where,link,category,tags
0,Analog,Analog is a modern American coffee shop locat...,: Daily 7 am-6 pm,235 Summit Ave E,https://www.instagram.com/p/Ca7pf4lJz7D/embed/...,Food & Drink,"Food & Drink,"
1,Union Coffee & Wine,Union is a great place to snag a cup of joe a...,: Daily 8 am-4 pm,"2407 E Union St Suite B, Seattle, WA",https://www.instagram.com/p/Cai82VIrvXE/embed/...,Food & Drink,"Food & Drink,"
2,General Porpoise,\nIf you want to enjoy a space that is beautif...,: Varies,Multiple locations,https://www.instagram.com/p/Cbp6tojJSfq/embed/...,Food & Drink,"Food & Drink,"
3,Hood Famous Bakeshop,Not only is Hood Famous super cute but they a...,": Wednesday-Friday 8 am-4 pm, Saturday & Sunda...","504 5th Ave S, Suite 107A",https://www.instagram.com/p/CZ7n5z3PGMj/embed/...,Food & Drink,"Food & Drink,"
4,Zeitgeist Coffee,This Pioneer Square spot is a classic coffee ...,: Monday-Friday 7 am-1 pm,171 S. Jackson Street,https://www.instagram.com/p/CJoPa8xgqeQ/embed/...,Food & Drink,"Food & Drink,"
...,...,...,...,...,...,...,...
734,Little Neon Taco,This Mexican taco spot is as modern as it is ...,,1011 Boren Ave,,Food & Drink,
735,Bar Melusine,"Clean lines, white walls, and dream decor mak...",,"1060 E Union St,",,Food & Drink,
736,Wilmott’s Ghost,When talking about beautiful restaurants in S...,,2100 6th Ave,,Food & Drink,
737,Tallulah’s,"Oh, go figure…it’s another restaurant from Th...",,550 19th Ave E,,Food & Drink,


In [329]:
result = naive_card_db_sea
result.to_json('~/Documents/INFO/Capstone22/recsys/food_drinks_seattle_card_X.json', orient='table')
result.to_csv('~/Documents/INFO/Capstone22/recsys/food_drinks_seattle_card_X.csv', index=True)

In [ ]:
# This is the first bot I crerated for Seattle data